## THETA FORECASTING

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import ta

from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models import ExponentialSmoothing, TBATS, AutoARIMA, Theta, RNNModel, NBEATSModel
from darts.metrics import mape, rmse

from darts.models import StatsForecastAutoTheta

In [3]:
df = pd.read_csv("../raw_data/20230609-170514-daily-processed-data.csv")

In [4]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [15]:
df

,timestamp,open,high,low,close,volume,volume_CMF,volume_FII,volume_MFI,volume_VWA,...,vortex_pos,S&P_close_filled,Interest_Rates_filled,Inflation_Rate_us,subhag_signal,Fisher,Fisher_trigger,supertrend,chop_index,Velocity
0,2019-09-12,10099.78,10184.66,10081.81,10161.51,621.690,0.008019,9381.841629,54.999476,10062.211552,...,1.084177,3009.57,2.25,1.70,1,1.05901,0.58994,1,55.107287,-1
1,2019-09-13,10321.45,10400.00,10321.45,10388.87,655.809,0.095988,11734.867436,77.615349,10259.840800,...,1.262003,3007.39,2.25,1.70,-1,2.53919,2.24085,1,42.784133,1
2,2019-09-14,10254.01,10299.66,10231.31,10287.11,681.268,0.033290,979.983540,41.709476,10275.093802,...,0.956061,3007.39,2.25,1.70,1,-1.52562,-1.92668,1,56.123524,-1
3,2019-09-15,10395.04,10419.97,10317.39,10377.71,1011.804,0.090962,4476.915550,46.417427,10338.125632,...,1.047632,3007.39,2.25,1.70,-1,2.34801,2.39448,1,49.875268,1
4,2019-09-16,10271.19,10304.56,10266.13,10289.03,834.126,0.021198,-752.692873,34.979972,10292.117754,...,0.947572,2997.96,2.25,1.70,-1,0.00179,0.18466,1,70.808813,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1357,2023-05-31,27745.70,27832.70,27637.40,27710.10,10281.328,-0.115085,-255844.295704,37.428422,27785.137663,...,0.967019,4179.83,5.25,5.15,1,-0.40399,-0.58092,1,58.137761,-1
1358,2023-06-01,27096.90,27108.80,27030.30,27081.90,5248.193,-0.065417,-468592.905790,30.792695,27028.789669,...,0.961442,4221.02,5.25,5.08,1,-0.02817,-0.35878,1,59.776339,-1
1359,2023-06-02,26850.50,26934.00,26849.00,26881.40,7588.041,-0.049525,-249646.914176,48.281993,26917.156581,...,1.044489,4282.37,5.25,5.08,-1,0.13181,0.46197,-1,52.103400,1
1360,2023-06-03,27159.40,27235.80,27132.00,27190.00,7451.957,0.038738,70819.873604,37.094107,27073.922697,...,1.050526,4282.37,5.25,5.08,1,1.55476,1.26044,1,56.726080,1


In [5]:
freq = '1d'
target_col = 'close'
time_col = 'timestamp'
series = TimeSeries.from_dataframe(df, time_col=time_col, value_cols=[target_col], freq=freq)

In [6]:
model = StatsForecastAutoTheta()

In [7]:
train, val = series.split_after(pd.Timestamp('2023-01-01'))

In [8]:
train

<TimeSeries (DataArray) (timestamp: 1208, component: 1, sample: 1)>
array([[[10161.51]],

       [[10388.87]],

       [[10287.11]],

       ...,

       [[16590.9 ]],

       [[16578.9 ]],

       [[16544.  ]]])
Coordinates:
  * timestamp  (timestamp) datetime64[ns] 2019-09-12 2019-09-13 ... 2023-01-01
  * component  (component) object 'close'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [9]:
val

<TimeSeries (DataArray) (timestamp: 154, component: 1, sample: 1)>
array([[[16597.6]],

       [[16751.9]],

       [[16658.8]],

       [[16819.4]],

       [[16840. ]],

       [[16922.9]],

       [[16931.5]],

       [[16949.2]],

       [[17172.5]],

       [[17453.6]],

...

       [[26465.9]],

       [[26711.4]],

       [[26759.4]],

       [[27828.5]],

       [[27653.4]],

       [[27710.1]],

       [[27081.9]],

       [[26881.4]],

       [[27190. ]],

       [[26990. ]]])
Coordinates:
  * timestamp  (timestamp) datetime64[ns] 2023-01-02 2023-01-03 ... 2023-06-04
  * component  (component) object 'close'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [13]:
len(val)

154

In [10]:
model.fit(train)
forecast = model.predict(1)
print("model {} obtains MAPE: {:.2f}%".format(model, mape(val[0], forecast)))

model StatsForecastAutoTheta() obtains MAPE: 0.29%


In [14]:
forecast

<TimeSeries (DataArray) (timestamp: 1, component: 1, sample: 1)>
array([[[16549.6796875]]])
Coordinates:
  * timestamp  (timestamp) datetime64[ns] 2023-01-02
  * component  (component) object 'close'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [11]:
forecast1 = model.predict(1)
print("Model {} obtains RMSE: {:.2f}".format(model, rmse(val[0], forecast1)))

Model StatsForecastAutoTheta() obtains RMSE: 47.92


In [ ]:
forecasts_list = []
# define a for loop for slicing the train set and make forecasts
for v in len(val):
    model= StatsForecastAutoTheta()
    model.fit(train)
    forecast = model.predict(v)


In [17]:
close1 = 1000
open1 = 950
high1= 1050
low1 = 900
prediction_2 = 910
atr_value= 30

In [ ]:
if close1> prediction_2:
    print("short trade")
    take_profit = (close1-prediction_2)/2
    stop_level = atr_value
for close in 